In [ ]:
# default_exp workflow
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

# Workflow

> Define static or dynamic workflow for automatically updating, training and deploying your ML model!


***input:*** optional

***output:*** python or snakemake script for running the workflow

***description:***

Define your automated workflow. You can either define static or dynamic workflow, or create a main loop routine and API for using your ML model.

Edit this and other text cells to describe your project.

## Import relevant modules

In [ ]:
# your code here

## Define notebook parameters

In [ ]:
# your code here

make direct derivations from the paramerters:

In [ ]:
# your code here

## Define your workflow

In [ ]:
# your code here